In [1]:
import pandas as pd

In [18]:
def read_table(rct):
    reg_table = pd.read_csv('../../Data/regression_table.csv')
    

In [19]:
def get_pareto_reward(year, policy_id, state):
    '''
    When Pareto decides a policy for a given state in a given year,
        This function will randomly select a valid experiment for the state
        And return its reward function
    '''    
    rct = get_rct(year, policy_id, state)
    
    reward = read_table(rct)
    
    return reward

In [44]:
def get_rct(year, policy_id, state):
    
    mapper = pd.read_csv('../../Data/Policy_ID_RCT_Table.csv')
    
    subset = mapper[ mapper['PolicyID'] == policy_id]

    if state in set(subset['Treatment State']):
        s2 = subset[ subset['Treatment State'] == state]['Control State'].as_matrix()[0]
        return [state, s2]
    
    else:
        reg_table = pd.read_csv('../../Data/regression_table.csv')
        # find a similar state that DID enact that policy
        # return them, along with the level of similarity 
    
        return 
    
rct = get_rct(2014, 3, 'Maine')
print (rct)


['Maine', 'West Virginia']


In [52]:
mapper = pd.read_csv('../../Data/Policy_ID_RCT_Table.csv')

subset = mapper[ mapper['PolicyID'] == 21]

state = 'Arkansas'

if state in set(subset['Treatment State']):
    s2 = subset[ subset['Treatment State'] == state]['Control State'].as_matrix()[0]
    rct =  [state, s2]

else:
    reg_table = pd.read_csv('../../Data/regression_table.csv')
    # find a similar state that DID enact that policy
    # return them, along with the level of similarity 
  

In [ ]:
# from the reg table, we need to pull all states that are similar
# we need to find 

In [46]:
reg_table

,State 1,State 2,Treatment Beta,Treatment T Value,Similarity T Value
0,Montana,South Carolina,1134.4206,17.550,0.139088
1,Montana,New York,584.0038,14.894,-0.388088
2,Montana,Ohio,1136.7439,25.722,0.220495
3,Montana,New Hampshire,917.8910,8.597,-0.257020
4,Arkansas,Montana,771.5403,10.384,-0.436243
5,Montana,Texas,629.7183,19.910,0.034430
6,Illinois,Montana,551.3692,11.993,0.090709
7,District of Columbia,Montana,1148.2508,8.123,0.644037
8,Connecticut,Montana,829.1544,10.088,0.518400
9,Montana,North Dakota,818.1128,6.732,0.105693
